In [1]:
# THIS DOESN'T CURRENTLY WORK WHEN TRYING TO RUN FROM BASH.

# This all only works from the root level folder.
import os
if 'py' not in os.listdir("."):
    os.chdir("..")

from loss_functions import loss_metrics
from optimize import optimize

import pandas as pd

AGE_CATEGORIES = ["senior", "child"]
DIS_CATEGORIES = ["dis_base"]
REGIONS = [
    "NORTH_EAST",
    "NORTH_WEST",
    "YORKSHIRE",
    "EAST_MIDLANDS",
    "WEST_MIDLANDS",
    "EAST_OF_ENGLAND",
    "LONDON",
    "SOUTH_EAST",
    "SOUTH_WEST",
    "WALES",
    "SCOTLAND",
    "NORTHERN_IRELAND",
]
categories = AGE_CATEGORIES + DIS_CATEGORIES + REGIONS

# Define bounds
AGE_BOUNDS = (40, 240)  # Child, working-age adult, senior.
DIS_BOUNDS = (0, 200)  # Base, severe, and enhanced disability UBI supplements.
GEO_BOUNDS = (-50, 50)  # Will be relative to a baseline geo.
# Skip adult which is calculated.
bounds = (
    [AGE_BOUNDS] * len(AGE_CATEGORIES)
    + [DIS_BOUNDS] * len(DIS_CATEGORIES)
    + [GEO_BOUNDS] * len(REGIONS)
)

input_dict = {category: bound for category, bound in zip(categories, bounds)}


def opt(reform):
    return optimize(
        input_dict,
        "mean_pct_loss",
        reform,
        verbose=False,
        seed=0,
        # Reforms don't always improve upon one another with the
        # default tolerance of 0.01.
        tol=0.0001,
    )


reform_1 = opt("reform_1")


Loss by all metrics:
 loser_share           4.483000e-01
losses                1.393906e+11
mean_pct_loss         4.980000e-02
mean_pct_loss_pwd2    5.630000e-02
gini                  2.704000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0498 

Optimal solution:
 adult               124
senior              207
child                76
dis_base              0
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64


In [2]:
reform_1

(     fun: 0.04984940366504809
  message: 'Optimization terminated successfully.'
     nfev: 2957
      nit: 12
  success: True
        x: array([206.69693634,  75.53401382,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ]),
 adult               124.036571
 senior              206.696936
 child                75.534014
 dis_base              0.000000
 NORTH_EAST            0.000000
 NORTH_WEST            0.000000
 YORKSHIRE             0.000000
 EAST_MIDLANDS         0.000000
 WEST_MIDLANDS         0.000000
 EAST_OF_ENGLAND       0.000000
 LONDON                0.000000
 SOUTH_EAST            0.000000
 SOUTH_WEST            0.000000
 WALES                 0.000000
 SCOTLAND              0.000000
 NORTHERN_IRELAND      0.000000
 dtype: float64)

In [4]:

reform_2 = opt("reform_2")
print(reform_2)

Loss by all metrics:
 loser_share           4.554000e-01
losses                1.410665e+11
mean_pct_loss         4.750000e-02
mean_pct_loss_pwd2    4.890000e-02
gini                  2.664000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0475 

Optimal solution:
 adult               119
senior              197
child                76
dis_base             74
NORTH_EAST            0
NORTH_WEST            0
YORKSHIRE             0
EAST_MIDLANDS         0
WEST_MIDLANDS         0
EAST_OF_ENGLAND       0
LONDON                0
SOUTH_EAST            0
SOUTH_WEST            0
WALES                 0
SCOTLAND              0
NORTHERN_IRELAND      0
dtype: int64
(     fun: 0.04747656235461934
 message: 'Optimization terminated successfully.'
    nfev: 3632
     nit: 15
 success: True
       x: array([197.00184467,  76.16426412,  73.76235325,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.    

In [5]:

reform_3 = opt("reform_3")


Loss by all metrics:
 loser_share           4.559000e-01
losses                1.404520e+11
mean_pct_loss         4.740000e-02
mean_pct_loss_pwd2    4.890000e-02
gini                  2.665000e-01
dtype: float64 

Optimal mean_pct_loss: 0.0474 

Optimal solution:
 adult               115
senior              192
child                72
dis_base             74
NORTH_EAST            6
NORTH_WEST            6
YORKSHIRE             0
EAST_MIDLANDS         4
WEST_MIDLANDS         4
EAST_OF_ENGLAND       2
LONDON                8
SOUTH_EAST            4
SOUTH_WEST            5
WALES                 3
SCOTLAND              5
NORTHERN_IRELAND      4
dtype: int64


NameError: name 'reform3' is not defined

In [10]:
print(reform_3)
# Check that iterations improve.
assert reform_2[0].fun < reform_1[0].fun, "Reform 2 doesn't improve on 1"
assert reform_3[0].fun < reform_2[0].fun, "Reform 3 doesn't improve on 2"

ubi_params = pd.DataFrame(reform_1[1]).T
ubi_params.loc[1] = reform_2[1]
ubi_params.loc[2] = reform_3[1]
ubi_params["mean_pct_loss"] = [
    reform_1[0].fun,
    reform_2[0].fun,
    reform_3[0].fun,
]
# Export to top-level folder.
ubi_params.to_csv("optimal_params.csv", index=False)
print("Parameters written!")

(     fun: 0.04741356811449964
 message: 'Optimization terminated successfully.'
    nfev: 13982
     nit: 61
 success: True
       x: array([196.69399838,  76.15827289,  73.81765765,   1.52944474,
         2.02220253,  -4.27135602,  -0.20821997,  -0.28111013,
        -2.36108423,   3.31055144,  -0.23312173,   0.93732727,
        -1.04016996,   0.2710138 ,   0.        ]), adult               114.852869
senior              192.422642
child                71.886917
dis_base             73.817658
NORTH_EAST            5.800801
NORTH_WEST            6.293559
YORKSHIRE             0.000000
EAST_MIDLANDS         4.063136
WEST_MIDLANDS         3.990246
EAST_OF_ENGLAND       1.910272
LONDON                7.581907
SOUTH_EAST            4.038234
SOUTH_WEST            5.208683
WALES                 3.231186
SCOTLAND              4.542370
NORTHERN_IRELAND      4.271356
dtype: float64)
Parameters written!


In [9]:
ls ..

analysis-template/  covid19/         min-wage/           ubicenter.org/
black_ubi/          covid_ui/        optimal_params.csv  ubicenter.py/
blog/               dynamic/         pfd_spm/            ubi-us-states/
blog-fp-1/          ed_debt_vs_ubi/  plan-explorer/      uk/
child-allowance/    emails/          plotly-converter/   UK-FT-UBI/
covid/              hanna_olken/     ubicenter.dev/      us-calc/


In [7]:
ubi_params

,adult,senior,child,dis_base,NORTH_EAST,NORTH_WEST,YORKSHIRE,EAST_MIDLANDS,WEST_MIDLANDS,EAST_OF_ENGLAND,LONDON,SOUTH_EAST,SOUTH_WEST,WALES,SCOTLAND,NORTHERN_IRELAND,mean_pct_loss
0,124.036571,206.696936,75.534014,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.049849
1,119.261503,197.001845,76.164264,73.762353,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.047477
2,114.852869,192.422642,71.886917,73.817658,5.800801,6.293559,0.0,4.063136,3.990246,1.910272,7.581907,4.038234,5.208683,3.231186,4.54237,4.271356,0.047414
